In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from skimage.io import imshow
from pathlib import Path
import pandas as pd
import time
import matplotlib.pyplot as plt
from keras.utils import plot_model
from keras.utils.vis_utils import *
import keras
import importlib
import pydot
from IPython.display import Image 

Using TensorFlow backend.


In [2]:
dataset_root = Path('../MURA-v1.1')

In [3]:
df = pd.read_csv(dataset_root/'train_image_paths.csv', header=None, names=['filename'])
df.head()

,filename
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...


In [4]:
df['class'] = (df.filename
               .str.extract('study.*_(positive|negative)'))
df.head()

,filename,class
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,positive
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,positive


In [5]:
def generate_df(dataset_root, csv_name):
    df = pd.read_csv(dataset_root/csv_name, header=None, names=['filename'])
    df['class'] = (df.filename
               .str.extract('study.*_(positive|negative)'))
    return df

In [6]:
list(dataset_root.parent.iterdir())

[PosixPath('../Notebooks 2019-20'),
 PosixPath('../.DS_Store'),
 PosixPath('../MURA-v1.1'),
 PosixPath('../DeepLearning2.ipynb'),
 PosixPath('../exec2'),
 PosixPath('../fashion-mnist-image-classification'),
 PosixPath('../.ipynb_checkpoints'),
 PosixPath('../Lectures 2019-20')]

In [18]:
batch_size=32
img_height=224
img_width=224
nb_epochs = 10

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1. / 255,
                                                          shear_range=0.2,
                                                          zoom_range=0.2,
                                                          rotation_range = 30, 
                                                          fill_mode = "nearest", 
                                                          width_shift_range = 0.1, 
                                                          height_shift_range = 0.1,
                                                          horizontal_flip=True,
                                                          validation_split=0.1)


train_generator = datagen.flow_from_dataframe(
    generate_df(dataset_root, 'train_image_paths.csv'),
    directory=dataset_root.parent,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary', 
    subset='training') # set as training data

validation_generator = datagen.flow_from_dataframe(
    generate_df(dataset_root, 'train_image_paths.csv'), # same directory as training data
    directory=dataset_root.parent,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

test_generator = datagen.flow_from_dataframe(generate_df(dataset_root, 'valid_image_paths.csv'),
    directory=dataset_root.parent,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 33128 images belonging to 2 classes.
Found 3680 images belonging to 2 classes.
Found 3197 images belonging to 2 classes.


In [20]:
def plotAccuracyLoss(someHistory):
    plt.style.use('dark_background')
    plt.rcParams['font.size'] = 14
    plt.figure(1, figsize = (15,8)) 
    plt.subplot(221)  
    plt.plot(someHistory.history['acc'])  
    plt.plot(someHistory.history['val_acc'])  
    plt.title('Model Accuracy')  
    plt.ylabel('Accuracy')  
    plt.xlabel('Epochs')  
    plt.legend(['train', 'valid'], loc='upper left')

    plt.subplot(222)  
    plt.plot(someHistory.history['loss'])  
    plt.plot(someHistory.history['val_loss'])  
    plt.title('Model Loss')  
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend(['train', 'valid'], loc='upper left')

    plt.show()

In [21]:
# Early stopping & checkpointing the best model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = 3)
cb_checkpointer = ModelCheckpoint(filepath = 'output/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

## DenseNet 50

In [ ]:
keras.__version__

'2.2.4'

In [ ]:
from keras.layers import *
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50

In [ ]:
resNet = ResNet50(
    weights= 'imagenet',
    include_top = False, 
    input_shape=(img_height, img_width, 3))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
resNet.trainable = False

In [ ]:
modelCNN2 = Sequential([
    resNet,
    Flatten(),
    Dense(1, activation='sigmoid')
])
modelCNN2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 100353    
Total params: 23,688,065
Trainable params: 100,353
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
modelCNN2.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.samples//batch_size
STEP_SIZE_VALID=validation_generator.samples//batch_size

t_start = time.time()

historyCNN2 = modelCNN2.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=nb_epochs,
                    callbacks=[cb_checkpointer, cb_early_stopper],
                    use_multiprocessing=True
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1035/1035 [==============================] - 18134s 18s/step - loss: 5.5165 - acc: 0.6574 - val_loss: 15.4304 - val_acc: 0.0427
Epoch 2/10
1035/1035 [==============================] - 10871s 11s/step - loss: 5.5255 - acc: 0.6572 - val_loss: 15.4304 - val_acc: 0.0427
Epoch 3/10
1035/1035 [==============================] - 10811s 10s/step - loss: 5.5168 - acc: 0.6577 - val_loss: 15.4304 - val_acc: 0.0427
Epoch 4/10
1035/1035 [==============================] - 10763s 10s/step - loss: 5.5279 - acc: 0.6570 - val_loss: 15.4304 - val_acc: 0.0427
Epoch 5/10
1035/1035 [==============================] - 10740s 10s/step - loss: 5.5245 - acc: 0.6572 - val_loss: 15.4304 - val_acc: 0.0427
Epoch 6/10
1035/1035 [==============================] - 12469s 12s/step - loss: 5.5226 - acc: 0.6574 - val_loss: 15.4304 - val_acc: 0.0427
Epoch 7/10
1035/1035 [==============================] - 13995s 14s/step - loss: 5.5299 - acc: 0.6569 - val_loss: 15.43

In [ ]:
modelCNN2.metrics_names

In [ ]:
modelCNN2.evaluate_generator(generator=test_generator, steps=STEP_SIZE_TRAIN)

In [ ]:
t_finish = time.time()
print(f"Kernel run time = {(t_finish-t_start)/3600} hours")

In [ ]:
print(historyCNN2.history.keys())

In [ ]:
plotAccuracyLoss(historyCNN2)